<a href="https://colab.research.google.com/github/evanphoward/AdventOfCode/blob/main/last_layer_dist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Imports
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import requests

In [ ]:
MNIST = True
TRAIN = False

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1,
               padding=1):
    '''
    A nn.Sequential layer executes its arguments in sequential order. In
    this case, it performs Conv2d -> BatchNorm2d -> ReLU. This is a typical
    block of layers used in Convolutional Neural Networks (CNNs). The 
    ConvNet implementation below stacks multiple instances of this three layer
    pattern in order to achieve over 90% classification accuracy on CIFAR-10.
    '''
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,
                  bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
        )

class ConvNet(nn.Module):
    '''
    A 9 layer CNN using the conv_block function above. Again, we use a
    nn.Sequential layer to build the entire model. The Conv2d layers get
    progressively larger (more filters) as the model gets deeper. This 
    corresponds to spatial resolution getting smaller (via the stride=2 blocks),
    going from 32x32 -> 16x16 -> 8x8. The nn.AdaptiveAvgPool2d layer at the end
    of the model reduces the spatial resolution from 8x8 to 1x1 using a simple
    average across all the pixels in each channel. This is then fed to the 
    single fully connected (linear) layer called classifier, which is the output
    prediction of the model.
    '''
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            conv_block(1 if MNIST else 3, 32),
            conv_block(32, 32),
            conv_block(32, 64, stride=2),
            conv_block(64, 64),
            conv_block(64, 64),
            conv_block(64, 128, stride=2),
            conv_block(128, 128),
            conv_block(128, 256),
            conv_block(256, 256),
            nn.AdaptiveAvgPool2d(1)
            )

        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        '''
        The forward function is called automatically by the model when it is
        given an input image. It first applies the 8 convolution layers, then
        finally the single classifier layer.
        '''
        h = self.model(x)
        B, C, _, _ = h.shape
        h = h.view(B, C)
        return self.classifier(h)


In [ ]:
model = ConvNet()
if MNIST:
  model.load_state_dict(torch.load("mnist_weights.pt"))
  np.save("mnist_classifier_weights.npy", model.classifier.weight.detach().numpy())
else:
  model.load_state_dict(torch.load("cifar_weights.pt"))
  np.save("cifar_classifier_weights.npy", model.classifier.weight.detach().numpy())

In [ ]:
NUM_DATA_PTS = 10000
if MNIST:
  transform = transforms.Compose([                                           
    transforms.ToTensor(),
    transforms.Normalize((0.1307), (0.3081)),
  ])
  dataset = torchvision.datasets.MNIST(root='./data', train=TRAIN, download=True, transform=transform)
else:
  transform = transforms.Compose([                                           
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])
  dataset = torchvision.datasets.CIFAR10(root='./data', train=TRAIN, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(dataset, range(NUM_DATA_PTS)), batch_size=NUM_DATA_PTS, num_workers=2)

model.classifier = nn.Identity()
model.eval()

with torch.no_grad():
  last_layer_inputs = []
  ground_truth_labels = []
  for inputs, targets in dataloader:
    last_layer_inputs.extend(model(inputs))
    ground_truth_labels.extend(targets)

  last_layer_inputs_tensor = torch.Tensor(len(last_layer_inputs), len(last_layer_inputs[0]))
  for i, last_layer_input in enumerate(last_layer_inputs):
    last_layer_inputs_tensor[i] = last_layer_input

In [ ]:
if MNIST:
  np.save("mnist_input_values.npy" if TRAIN else "mnist_input_values_test.npy", last_layer_inputs_tensor.numpy())
  np.save("mnist_ground_labels.npy" if TRAIN else "mnist_ground_labels_test.npy", np.array(ground_truth_labels))
else:
  np.save("cifar_input_values.npy" if TRAIN else "cifar_input_values_test.npy", last_layer_inputs_tensor.numpy())
  np.save("cifar_ground_labels.npy" if TRAIN else "cifar_ground_labels_test.npy", np.array(ground_truth_labels))